In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install nltk scikit-learn pandas

In [3]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import NMF
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [20]:
#df = pd.read_csv('/content/drive/MyDrive/concatenated_data.csv')
X_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/test/X_test.csv')
X_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/train/X_train.csv')

In [21]:
X_train= X_train.dropna()
X_test= X_test.dropna()

In [22]:
y_train = X_train['main_category']
X_train = X_train.drop('main_category', axis=1)


y_test = X_test['main_category']
X_test = X_test.drop('main_category', axis=1)

In [23]:
train_documents = X_train['description'].tolist()
test_documents = X_test['description'].tolist()

train_category = y_train.tolist()
test_category = y_test.tolist()

In [24]:
# Vectorize the training data
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
X_train = vectorizer.fit_transform(train_documents)
X_test = vectorizer.transform(test_documents)


In [25]:
# Encode the category labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_category)
y_test = label_encoder.transform(test_category)


In [26]:

nmf = NMF(n_components=5, random_state=42, max_iter=200, solver='mu', beta_loss='frobenius')

# Logistic regression classifier to map NMF topics to categories
classifier = LogisticRegression(random_state=42, max_iter=1000)

nmf.fit(X_train, y_train)

NMF(n_components=5, random_state=42, solver='mu')

In [28]:
# Get the top words for each topic
def display_topics(model, feature_names, num_top_words):
    # nmf_model = model.named_steps['nmf']
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}:")
        print(", ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))
        print()

display_topics(nmf, vectorizer.get_feature_names_out(), 10)


Topic 1:
usb, case, camera, feature, cable, flag, power, material, plant, battery

Topic 2:
hair, wig, clip, comb, shampoo, brush, conditioner, style, human, dry

Topic 3:
nail, polish, gel, art, apply, coat, file, sticker, glue, manicure

Topic 4:
skin, oil, ingredient, body, organic, flavor, tea, oz, cream, chocolate

Topic 5:
book, author, life, review, story, university, novel, world, read, history



In [29]:
#training the classfier
classifier.fit(nmf.transform(X_train), y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [31]:
from sklearn.metrics import classification_report

# Predict categories for the test set
predicted_categories = classifier.predict(nmf.transform(X_test))
predicted_categories = label_encoder.inverse_transform(predicted_categories)

accuracy = accuracy_score(test_category, predicted_categories)

# Evaluate the model
print(classification_report(test_category, predicted_categories))


              precision    recall  f1-score   support

      beauty       0.77      0.70      0.73     12703
       books       0.99      0.98      0.98      9723
 electronics       0.61      0.72      0.66      9169
     grocery       0.68      0.72      0.70      9022
        home       0.53      0.46      0.49      9044

    accuracy                           0.72     49661
   macro avg       0.71      0.72      0.71     49661
weighted avg       0.72      0.72      0.72     49661



In [51]:
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models

In [53]:
# X_test_tokens = [str(doc).split() for doc in test_documents]
test_documents_tokens = [doc.split() for doc in test_documents]

In [54]:
words = vectorizer.get_feature_names_out()
#since the CoherenceModel didn't work initially with nmf we needed to extract the topics manually
num_top_words = 10  # Number of top words to display for each topic
topics = []

for topic_idx, topic in enumerate(nmf.components_):
    top_words_idx = topic.argsort()[:-num_top_words - 1:-1]  # Get indices of the top words
    top_words = [words[i] for i in top_words_idx]  # Map indices to words
    topics.append(top_words)


dictionary = corpora.Dictionary(test_documents_tokens)

In [55]:
coherence_model_nmf = CoherenceModel(topics=topics, texts=test_documents_tokens, dictionary=dictionary, coherence='c_v')
coherence_nmf = coherence_model_nmf.get_coherence()
print(f"Coherence Score: {coherence_nmf}")

Coherence Score: 0.721909509714809


In [56]:
import joblib
joblib.dump(nmf, 'nmf_Model.joblib')

['nmf_Model.joblib']

In [57]:
# Predict category for a new product description
def predict_category(description, vectorizer, model, label_encoder):
    vectorized = vectorizer.transform([description])
    category_index = model.predict(vectorized)[0]
    return label_encoder.inverse_transform([category_index])[0]

# Example
new_description = "good harts by the auther maha was new york best seller"
predicted_category = predict_category(new_description, vectorizer, model, label_encoder)
print(f"Predicted Category: {predicted_category}")

Predicted Category: books
